### data process
features: embeddings

labels: pulmonary, race

### model
train for pulmonary
train for race

### analyze
look at performance by race ?

In [1]:
from google.colab import auth
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
import os
import tensorflow_datasets as tfds
from sklearn.decomposition import PCA
import os
from google.cloud import storage

KeyboardInterrupt: ignored

In [ ]:
auth.authenticate_user()

In [ ]:
df_admissions = pd.read_csv('/content/admissions.csv.gz')[['subject_id', 'insurance', 'race']]

In [ ]:
df_admissions

,subject_id,insurance,race
0,10000032,Other,WHITE
1,10000032,Medicaid,WHITE
2,10000032,Medicaid,WHITE
3,10000032,Medicaid,WHITE
4,10000068,Other,WHITE
...,...,...,...
431226,19999828,Other,WHITE
431227,19999828,Other,WHITE
431228,19999840,Other,WHITE
431229,19999840,Other,WHITE


In [ ]:
# from tutorial
def download_blob(bucket, source_blob_name: str, destination_file_name: str):
    """
    Downloads a blob from the bucket.
    
    https://cloud.google.com/storage/docs/downloading-objects
    """
    blob = bucket.blob(source_blob_name)
    try:
      blob.download_to_filename(destination_file_name)
    except Exception as e:
      print('Error during download - do you have the right permissions?')
      print(e)
      return
    print(f"Downloaded {source_blob_name}")


storage_client = storage.Client()

# Make a directory to download the data
if not os.path.exists('data'):
  os.mkdir('data')

In [ ]:
embeddings_bucket = storage_client.bucket(
    'image-embeddings-mimic-cxr-1.0.physionet.org')    

# Download the checksums file which contains a records list
download_blob(embeddings_bucket, "SHA256SUMS.txt", "data/SHA256SUMS.txt")

Downloaded SHA256SUMS.txt


In [ ]:
df_embeddings = pd.read_csv("data/SHA256SUMS.txt", delimiter=" ", header=None, skiprows=[0])  # Skip the license file entry
display(df_embeddings.head())

,0,1
0,70e8c87d38d5dfba586218ccafb7428f76a3c7b08f6179...,files/p10/p10000032/s50414267/02aa804e-bde0afd...
1,a0c7a01d9fc223b25767ebf04d6f736ac4cca1f9336946...,files/p10/p10000032/s53189527/2a2277a9-b0ded15...
2,38202feabe98d8cd042b50879a886c8a099aba9afef18c...,files/p10/p10000032/s53911762/68b5c4b1-227d048...
3,23283d7a2cb72d8e59f21dfd33d66be9082f69b18c5c81...,files/p10/p10000032/s53911762/fffabebf-74fd3a1...
4,c62e51c914c91b2bc2d3c14a82046709b800bada664e5c...,files/p10/p10000032/s56699142/ea030e7a-2e3b134...


In [ ]:
import re

# Example: 'files/p19/p19692222/s59566639/965b6053-a2c70d67-c0467ca6-02372346-fb7c6224.tfrecord'
FILE_PATTERN = re.compile(r"files/(?:\w+)/p(?P<subject_id>\w+)/s(?P<study_id>\w+)/(?P<dicom_id>[\w-]+)\.tfrecord")

def parse_file_pattern(file_path: str):
    """
    Extracts the subject_id, study_id, and dicom_id
    from the full file path string.
    """
    match = FILE_PATTERN.fullmatch(file_path)
    if not match:
        raise Exception(f"Failed to match file path: {file_path}")
    return (int(match[1]), int(match[2]), match[3])

# Create additional columns from file path components
df_embeddings = df_embeddings[[1]]
df_embeddings.rename(columns={1: "embedding_file"}, inplace=True)
df_embeddings[["subject_id","study_id", "dicom_id"]] = df_embeddings.apply(
    lambda x: parse_file_pattern(x["embedding_file"]), axis=1, result_type="expand")

display(df_embeddings)

<ipython-input-13-04503baf66bf>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_embeddings.rename(columns={1: "embedding_file"}, inplace=True)


,embedding_file,subject_id,study_id,dicom_id
0,files/p10/p10000032/s50414267/02aa804e-bde0afd...,10000032,50414267,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014
1,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,10000032,53189527,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab
2,files/p10/p10000032/s53911762/68b5c4b1-227d048...,10000032,53911762,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714
3,files/p10/p10000032/s53911762/fffabebf-74fd3a1...,10000032,53911762,fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818
4,files/p10/p10000032/s56699142/ea030e7a-2e3b134...,10000032,56699142,ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c
...,...,...,...,...
243319,files/p19/p19999733/s57132437/3fcd0406-9b11160...,19999733,57132437,3fcd0406-9b111603-feae7033-96632b3a-111333e5
243320,files/p19/p19999733/s57132437/428e2c18-5721d8f...,19999733,57132437,428e2c18-5721d8f3-35a05001-36f3d080-9053b83c
243321,files/p19/p19999987/s55368167/58766883-376a15c...,19999987,55368167,58766883-376a15ce-3b323a28-6af950a0-16b793bd
243322,files/p19/p19999987/s58621812/7ba273af-3d290f8...,19999987,58621812,7ba273af-3d290f8d-e28d0ab4-484b7a86-7fc12b08


# Embeddings download

In [ ]:
cxr_jpg_bucket = storage_client.bucket(
    'mimic-cxr-jpg-2.0.0.physionet.org')

CXR_JPG_METADATA_FILES = (
    "mimic-cxr-2.0.0-metadata.csv.gz",
    "mimic-cxr-2.0.0-split.csv.gz",
    "mimic-cxr-2.0.0-chexpert.csv.gz")

for fname in CXR_JPG_METADATA_FILES:
  download_blob(cxr_jpg_bucket, fname, f"data/{fname}")

Downloaded mimic-cxr-2.0.0-metadata.csv.gz
Downloaded mimic-cxr-2.0.0-split.csv.gz
Downloaded mimic-cxr-2.0.0-chexpert.csv.gz


In [ ]:
df_metadata = pd.read_csv(f"data/{CXR_JPG_METADATA_FILES[0]}", compression="gzip")
df_split = pd.read_csv(f"data/{CXR_JPG_METADATA_FILES[1]}", compression="gzip")
df_labels_chexpert = pd.read_csv(f"data/{CXR_JPG_METADATA_FILES[2]}", compression="gzip")

display(df_metadata.head())
display(df_split.head())
display(df_labels_chexpert.head())

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN


,dicom_id,study_id,subject_id,split
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,50414267,10000032,train
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,50414267,10000032,train
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,53189527,10000032,train
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,53189527,10000032,train
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,53911762,10000032,train


,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,10000032,53911762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,10000032,56699142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,10000764,57375967,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN


# Create Labels file

In [ ]:
# Each study contains one or more DICOMs
# Chexpert labels df does not contain DICOM ID. Must join on (subject_id + study_id)
df_labels = df_split.merge(df_labels_chexpert, on=['subject_id', 'study_id'])
df_labels = df_labels.merge(df_metadata, on=['dicom_id'])
df_labels = df_embeddings.merge(df_labels, on=['dicom_id'], how='left')
#df_labels['insurance'] = df_labels.apply(lambda x: df_admissions[df_admissions['subject_id']==x['subject_id']].head(1)['insurance'], axis=1)

# df['c'] = df.apply(lambda x: max(len(x['a']), len(x['b'])), axis=1)

display(df_labels.head())

,embedding_file,subject_id,study_id,dicom_id,study_id_x,subject_id_x,split,Atelectasis,Cardiomegaly,Consolidation,...,study_id_y,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning
0,files/p10/p10000032/s50414267/02aa804e-bde0afd...,10000032,50414267,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,50414267,10000032,train,NaN,NaN,NaN,...,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect
1,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,10000032,53189527,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,53189527,10000032,train,NaN,NaN,NaN,...,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect
2,files/p10/p10000032/s53911762/68b5c4b1-227d048...,10000032,53911762,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,53911762,10000032,train,NaN,NaN,NaN,...,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN
3,files/p10/p10000032/s53911762/fffabebf-74fd3a1...,10000032,53911762,fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818,53911762,10000032,train,NaN,NaN,NaN,...,53911762,CHEST (PORTABLE AP),AP,2906,2258,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,Erect
4,files/p10/p10000032/s56699142/ea030e7a-2e3b134...,10000032,56699142,ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c,56699142,10000032,train,NaN,NaN,NaN,...,56699142,CHEST (PORTABLE AP),AP,3056,2544,21800805,234424.765,CHEST (PORTABLE AP),antero-posterior,NaN


In [ ]:
agg_df = df_admissions.groupby('subject_id').aggregate({'insurance': list})
agg_df['label'] = agg_df.apply(lambda x: 1 if 'Medicaid' in x['insurance'] else 0, axis=1)
agg_df

,insurance,label
subject_id,,
10000032,"[Other, Medicaid, Medicaid, Medicaid]",1
10000068,[Other],0
10000084,"[Medicare, Medicare]",1
10000108,[Other],0
10000117,"[Other, Other]",0
...,...,...
19999733,[Other],0
19999784,"[Other, Medicaid, Medicaid, Other, Other, Medi...",1
19999828,"[Other, Other]",0


In [ ]:
left_merged = pd.merge(df_labels, agg_df, how="left", on=["subject_id"])
left_merged = left_merged.drop(columns='insurance')
left_merged = left_merged.rename(columns={'label': 'insurance'})
left_merged

,embedding_file,subject_id,study_id,dicom_id,study_id_x,subject_id_x,split,Atelectasis,Cardiomegaly,Consolidation,...,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning,insurance
0,files/p10/p10000032/s50414267/02aa804e-bde0afd...,10000032,50414267,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,50414267,10000032,train,NaN,NaN,NaN,...,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect,1.0
1,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,10000032,53189527,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,53189527,10000032,train,NaN,NaN,NaN,...,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect,1.0
2,files/p10/p10000032/s53911762/68b5c4b1-227d048...,10000032,53911762,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,53911762,10000032,train,NaN,NaN,NaN,...,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN,1.0
3,files/p10/p10000032/s53911762/fffabebf-74fd3a1...,10000032,53911762,fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818,53911762,10000032,train,NaN,NaN,NaN,...,CHEST (PORTABLE AP),AP,2906,2258,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,Erect,1.0
4,files/p10/p10000032/s56699142/ea030e7a-2e3b134...,10000032,56699142,ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c,56699142,10000032,train,NaN,NaN,NaN,...,CHEST (PORTABLE AP),AP,3056,2544,21800805,234424.765,CHEST (PORTABLE AP),antero-posterior,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243319,files/p19/p19999733/s57132437/3fcd0406-9b11160...,19999733,57132437,3fcd0406-9b111603-feae7033-96632b3a-111333e5,57132437,19999733,train,NaN,NaN,NaN,...,CHEST (PA AND LAT),PA,3056,2544,21520708,224550.171,CHEST (PA AND LAT),postero-anterior,Erect,0.0
243320,files/p19/p19999733/s57132437/428e2c18-5721d8f...,19999733,57132437,428e2c18-5721d8f3-35a05001-36f3d080-9053b83c,57132437,19999733,train,NaN,NaN,NaN,...,CHEST (PA AND LAT),PA,3056,2544,21520708,224550.171,CHEST (PA AND LAT),postero-anterior,Erect,0.0
243321,files/p19/p19999987/s55368167/58766883-376a15c...,19999987,55368167,58766883-376a15ce-3b323a28-6af950a0-16b793bd,55368167,19999987,train,1.0,-1.0,NaN,...,CHEST (PORTABLE AP),AP,2544,3056,21451104,51448.218,CHEST (PORTABLE AP),antero-posterior,Erect,0.0
243322,files/p19/p19999987/s58621812/7ba273af-3d290f8...,19999987,58621812,7ba273af-3d290f8d-e28d0ab4-484b7a86-7fc12b08,58621812,19999987,train,1.0,NaN,NaN,...,CHEST (PORTABLE AP),AP,3056,2544,21451102,202809.234,CHEST (PORTABLE AP),antero-posterior,Erect,0.0


In [ ]:
agg_df = df_admissions.groupby('subject_id').aggregate({'race': list})
#agg_df['label'] = agg_df.apply(lambda x: 1 if 'Medicaid' in x['insurance'] or 'Medicare' in x['insurance'] else 0, axis=1)
agg_df["race1"] = agg_df["race"].str[0]
agg_df

,race,race1
subject_id,,
10000032,"[WHITE, WHITE, WHITE, WHITE]",WHITE
10000068,[WHITE],WHITE
10000084,"[WHITE, WHITE]",WHITE
10000108,[WHITE],WHITE
10000117,"[WHITE, WHITE]",WHITE
...,...,...
19999733,[WHITE],WHITE
19999784,"[BLACK/AFRICAN AMERICAN, BLACK/AFRICAN AMERICA...",BLACK/AFRICAN AMERICAN
19999828,"[WHITE, WHITE]",WHITE


In [ ]:
final_df = pd.merge(left_merged, agg_df, how="left", on=["subject_id"])
final_df = final_df.drop(columns=['race'])
final_df = final_df.rename(columns={'race1': 'race'})
final_df

,embedding_file,subject_id,study_id,dicom_id,study_id_x,subject_id_x,split,Atelectasis,Cardiomegaly,Consolidation,...,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning,insurance,race
0,files/p10/p10000032/s50414267/02aa804e-bde0afd...,10000032,50414267,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,50414267,10000032,train,NaN,NaN,NaN,...,PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect,1.0,WHITE
1,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,10000032,53189527,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,53189527,10000032,train,NaN,NaN,NaN,...,PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect,1.0,WHITE
2,files/p10/p10000032/s53911762/68b5c4b1-227d048...,10000032,53911762,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,53911762,10000032,train,NaN,NaN,NaN,...,AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN,1.0,WHITE
3,files/p10/p10000032/s53911762/fffabebf-74fd3a1...,10000032,53911762,fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818,53911762,10000032,train,NaN,NaN,NaN,...,AP,2906,2258,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,Erect,1.0,WHITE
4,files/p10/p10000032/s56699142/ea030e7a-2e3b134...,10000032,56699142,ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c,56699142,10000032,train,NaN,NaN,NaN,...,AP,3056,2544,21800805,234424.765,CHEST (PORTABLE AP),antero-posterior,NaN,1.0,WHITE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243319,files/p19/p19999733/s57132437/3fcd0406-9b11160...,19999733,57132437,3fcd0406-9b111603-feae7033-96632b3a-111333e5,57132437,19999733,train,NaN,NaN,NaN,...,PA,3056,2544,21520708,224550.171,CHEST (PA AND LAT),postero-anterior,Erect,0.0,WHITE
243320,files/p19/p19999733/s57132437/428e2c18-5721d8f...,19999733,57132437,428e2c18-5721d8f3-35a05001-36f3d080-9053b83c,57132437,19999733,train,NaN,NaN,NaN,...,PA,3056,2544,21520708,224550.171,CHEST (PA AND LAT),postero-anterior,Erect,0.0,WHITE
243321,files/p19/p19999987/s55368167/58766883-376a15c...,19999987,55368167,58766883-376a15ce-3b323a28-6af950a0-16b793bd,55368167,19999987,train,1.0,-1.0,NaN,...,AP,2544,3056,21451104,51448.218,CHEST (PORTABLE AP),antero-posterior,Erect,0.0,UNKNOWN
243322,files/p19/p19999987/s58621812/7ba273af-3d290f8...,19999987,58621812,7ba273af-3d290f8d-e28d0ab4-484b7a86-7fc12b08,58621812,19999987,train,1.0,NaN,NaN,...,AP,3056,2544,21451102,202809.234,CHEST (PORTABLE AP),antero-posterior,Erect,0.0,UNKNOWN


In [ ]:
final_df.drop(columns=['split'])
final_df

,embedding_file,subject_id,study_id,dicom_id,study_id_x,subject_id_x,split,Atelectasis,Cardiomegaly,Consolidation,...,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning,insurance,race
0,files/p10/p10000032/s50414267/02aa804e-bde0afd...,10000032,50414267,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,50414267,10000032,train,NaN,NaN,NaN,...,PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect,1.0,WHITE
1,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,10000032,53189527,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,53189527,10000032,train,NaN,NaN,NaN,...,PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect,1.0,WHITE
2,files/p10/p10000032/s53911762/68b5c4b1-227d048...,10000032,53911762,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,53911762,10000032,train,NaN,NaN,NaN,...,AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN,1.0,WHITE
3,files/p10/p10000032/s53911762/fffabebf-74fd3a1...,10000032,53911762,fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818,53911762,10000032,train,NaN,NaN,NaN,...,AP,2906,2258,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,Erect,1.0,WHITE
4,files/p10/p10000032/s56699142/ea030e7a-2e3b134...,10000032,56699142,ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c,56699142,10000032,train,NaN,NaN,NaN,...,AP,3056,2544,21800805,234424.765,CHEST (PORTABLE AP),antero-posterior,NaN,1.0,WHITE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243319,files/p19/p19999733/s57132437/3fcd0406-9b11160...,19999733,57132437,3fcd0406-9b111603-feae7033-96632b3a-111333e5,57132437,19999733,train,NaN,NaN,NaN,...,PA,3056,2544,21520708,224550.171,CHEST (PA AND LAT),postero-anterior,Erect,0.0,WHITE
243320,files/p19/p19999733/s57132437/428e2c18-5721d8f...,19999733,57132437,428e2c18-5721d8f3-35a05001-36f3d080-9053b83c,57132437,19999733,train,NaN,NaN,NaN,...,PA,3056,2544,21520708,224550.171,CHEST (PA AND LAT),postero-anterior,Erect,0.0,WHITE
243321,files/p19/p19999987/s55368167/58766883-376a15c...,19999987,55368167,58766883-376a15ce-3b323a28-6af950a0-16b793bd,55368167,19999987,train,1.0,-1.0,NaN,...,AP,2544,3056,21451104,51448.218,CHEST (PORTABLE AP),antero-posterior,Erect,0.0,UNKNOWN
243322,files/p19/p19999987/s58621812/7ba273af-3d290f8...,19999987,58621812,7ba273af-3d290f8d-e28d0ab4-484b7a86-7fc12b08,58621812,19999987,train,1.0,NaN,NaN,...,AP,3056,2544,21451102,202809.234,CHEST (PORTABLE AP),antero-posterior,Erect,0.0,UNKNOWN


In [ ]:

final_df.to_csv('data-labelled', encoding='utf-8')

insurance is 1 for medicaid/medicare, 0 for private